In [5]:
import pandas as pd
import openpyxl

#  load the file without skipping rows first
df = pd.read_excel("/home/ggirelli/Documents/DataAnalysis/Datasets/Official_Open-Datasets_Aruba_CBS/Table-1.9-Relative-population-by-age-groups-in-percentage.xlsx")

# extract the correct header rows
# row 0 has the Years (2015, NaN, NaN...)
# row 1 has the Categories (Male, Female, Total...)
years = df.iloc[0].ffill() 
categories = df.iloc[1]

# glue them together
new_headers = []
for y, c in zip(years, categories):
    if pd.isna(c):
        new_headers.append(y) # Keep 'Age' as 'Age'
    else:
        new_headers.append(f"{y}_{c}")

# apply the new headers
df.columns = new_headers

# drop the first two rows (which are now our headers) 
# and keep only the three main age groups (Rows 2, 3, and 40
df_clean = df.iloc[2:5].copy()

# melt all columns except "Age"
df_long = df_clean.melt(id_vars = ["Age"], var_name = "Year_Sex", value_name = "Percentage")

# split "2015_Male" into two separate columns: "Year" and "Sex"
# expand = True puts the result into two new columns
df_long[["Year", "Sex"]] =  df_long["Year_Sex"].str.split('_', expand =  True)

# final polish
df_long["Year"] =  df_long["Year"].astype(int)
df_final = df_long[["Year", "Age", "Sex", "Percentage"]].rename(columns = {"Age": "Age_Group"})

print(df_final.head(9))

   Year Age_Group     Sex Percentage
0  2015    0 - 14    Male        9.6
1  2015   15 - 59    Male       29.4
2  2015      60 +    Male          8
3  2015    0 - 14  Female        9.2
4  2015   15 - 59  Female       32.8
5  2015      60 +  Female       10.6
6  2015    0 - 14   Total       18.9
7  2015   15 - 59   Total       62.3
8  2015      60 +   Total       18.7
